# S&P500, Nasdaq, USD, Oil, Gold, Gas

In [5]:
from cmath import nan
import pandas as pd
import datetime
import time
import requests
import numpy as np
import os
import yfinance as yf

# 연속 로그 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
def sort_by_date_diff_series_data(df_base, col):
    diff_col = f"DIFF_{col}"
    df_base = df_base.sort_values(by=["KOR_DT"])    
    df_base[diff_col] = df_base[col].diff()
    return df_base.dropna()

In [13]:
# Kodex 데이터 크롤링
list_columns_kr = [
    "KOR_DT",
    "Closing_Price", "Start_Price", "High_Price", "Low_Price", "Volume"
]
dict_rename_columns = {
    "종가": "Closing_Price", 
    "시가": "Start_Price", 
    "고가": "High_Price", 
    "저가": "Low_Price", 
    "거래량": "Volume",
}

cd = "122630"
list_df = []
for idx in range(25):
    pg = idx + 1
    url = f"https://finance.naver.com/item/sise_day.naver?code={cd}&page={pg}"
    headers = {
        "referer" : url,    
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
    }
    res = requests.get(url=url, headers=headers)
    list_df.append(pd.read_html(res.text)[0].dropna())

df_kodex = pd.concat(list_df)
df_kodex["KOR_DT"] = df_kodex["날짜"].apply(lambda X: X.replace(".","-"))

df_kodex = df_kodex.rename(columns=dict_rename_columns)
df_kodex["KOR_DT"] = pd.to_datetime(df_kodex["KOR_DT"])
df_kodex = df_kodex[list_columns_kr]

for col in dict_rename_columns.values():
    df_kodex = sort_by_date_diff_series_data(df_kodex, col)
df_kodex

,KOR_DT,Closing_Price,Start_Price,High_Price,Low_Price,Volume,DIFF_Closing_Price,DIFF_Start_Price,DIFF_High_Price,DIFF_Low_Price,DIFF_Volume
5,2021-06-08,28490.0,28575.0,28740.0,28405.0,15550340.0,-100.0,-185.0,-110.0,160.0,-4673209.0
4,2021-06-09,27880.0,28420.0,28420.0,27880.0,16971807.0,-610.0,-155.0,-320.0,-525.0,1421467.0
3,2021-06-10,28055.0,27895.0,28220.0,27805.0,13855404.0,175.0,-525.0,-200.0,-75.0,-3116403.0
2,2021-06-11,28450.0,28170.0,28550.0,28165.0,19567411.0,395.0,275.0,330.0,360.0,5712007.0
1,2021-06-14,28455.0,28505.0,28510.0,28250.0,10911607.0,5.0,335.0,-40.0,85.0,-8655804.0
...,...,...,...,...,...,...,...,...,...,...,...
5,2022-05-27,18370.0,18400.0,18495.0,18295.0,18914687.0,395.0,315.0,30.0,455.0,-1678204.0
4,2022-05-30,18845.0,18750.0,18870.0,18590.0,21093524.0,475.0,350.0,375.0,295.0,2178837.0
3,2022-05-31,18995.0,18820.0,19055.0,18600.0,18266961.0,150.0,70.0,185.0,10.0,-2826563.0
2,2022-06-02,18575.0,18735.0,18815.0,18500.0,18195822.0,-420.0,-85.0,-240.0,-100.0,-71139.0


In [7]:
list_columns_us = [
    "KOR_DT",
    "S&P500_adj_prc", "Nasdaq_adj_prc", "CrudeOil_adj_prc", "Gold_adj_prc", "10-YrBond_adj_prc",
]

dict_jongmok_us = {
    "S&P500": "^GSPC",
    "Nasdaq": "^IXIC",
    "Gold": "GC=F",
    "CrudeOil": "CL=F",
    "10-YrBond": "^TNX",
}

start_dt = '2020-06-01'
df_us_index = pd.DataFrame(columns=["Date"])
for nm, cd in dict_jongmok_us.items():
    print(nm, cd)
    df = yf.download(cd,start = start_dt).reset_index()
    adj_close_nm = f"{nm}_adj_prc"
    df = df.rename(columns={"Adj Close": adj_close_nm})[["Date", adj_close_nm]]
    df_us_index = pd.merge(
        df, df_us_index, how="left", on="Date"
    )
    
df_us_index["KOR_DT"] = df_us_index["Date"].apply(lambda X: X + datetime.timedelta(days=1))
df_us_index = df_us_index[list_columns_us]

for col in list_columns_us:
    if col == "KOR_DT": continue
    df_us_index = sort_by_date_diff_series_data(df_us_index, col)
df_us_index

S&P500 ^GSPC
[*********************100%***********************]  1 of 1 completed
Nasdaq ^IXIC
[*********************100%***********************]  1 of 1 completed
Gold GC=F
[*********************100%***********************]  1 of 1 completed
CrudeOil CL=F
[*********************100%***********************]  1 of 1 completed
10-YrBond ^TNX
[*********************100%***********************]  1 of 1 completed


,KOR_DT,S&P500_adj_prc,Nasdaq_adj_prc,CrudeOil_adj_prc,Gold_adj_prc,10-YrBond_adj_prc,DIFF_S&P500_adj_prc,DIFF_Nasdaq_adj_prc,DIFF_CrudeOil_adj_prc,DIFF_Gold_adj_prc,DIFF_10-YrBond_adj_prc
5,2020-06-09,3232.389893,9924.750000,38.189999,1698.300049,0.884,38.459961,110.669922,-1.360001,22.100098,-0.020
6,2020-06-10,3207.179932,9953.750000,38.939999,1714.699951,0.829,-25.209961,29.000000,0.750000,16.399902,-0.055
7,2020-06-11,3190.139893,10020.349609,39.599998,1713.300049,0.748,-17.040039,66.599609,0.660000,-1.399902,-0.081
8,2020-06-12,3002.100098,9492.730469,36.340000,1732.000000,0.653,-188.039795,-527.619141,-3.259998,18.699951,-0.095
9,2020-06-13,3041.310059,9588.809570,36.259998,1729.300049,0.699,39.209961,96.079102,-0.080002,-2.699951,0.046
...,...,...,...,...,...,...,...,...,...,...,...
503,2022-05-28,4158.240234,12131.129883,115.070000,1851.300049,2.743,100.400146,390.479492,0.980003,3.900024,-0.013
504,2022-06-01,4132.149902,12081.389648,114.669998,1842.699951,2.844,-26.090332,-49.740234,-0.400002,-8.600098,0.101
505,2022-06-02,4101.229980,11994.459961,115.260002,1843.300049,2.931,-30.919922,-86.929688,0.590004,0.600098,0.087
506,2022-06-03,4176.819824,12316.900391,116.870003,1866.500000,2.913,75.589844,322.440430,1.610001,23.199951,-0.018


In [8]:
list_df = []
for idx in range(50):
    pg = idx + 1
    url = f"https://finance.naver.com/marketindex/exchangeDailyQuote.naver?marketindexCd=FX_USDKRW&page={pg}"
    headers = {
        "referer" : url,    
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
    }
    res = requests.get(url=url, headers=headers)
    list_df.append(pd.read_html(res.text, header=[1])[0][["날짜", "매매기준율"]])

df_usd = pd.concat(list_df)
df_usd["KOR_DT"] = df_usd["날짜"].apply(lambda X: X.replace(".","-"))
df_usd["KOR_DT"] = pd.to_datetime(df_usd["KOR_DT"])
df_usd["KOR_DT"] = df_usd["KOR_DT"].apply(lambda X: X + datetime.timedelta(days=1))
df_usd = df_usd.rename(columns=({"매매기준율": "krw_usd"}))

df_usd = sort_by_date_diff_series_data(df_usd, "krw_usd")
df_usd

,날짜,krw_usd,KOR_DT,DIFF_krw_usd
8,2020.06.02,1222.5,2020-06-03,-5.5
7,2020.06.03,1217.0,2020-06-04,-5.5
6,2020.06.04,1219.5,2020-06-05,2.5
5,2020.06.05,1209.0,2020-06-06,-10.5
4,2020.06.08,1201.0,2020-06-09,-8.0
...,...,...,...,...
4,2022.05.27,1256.0,2022-05-28,-9.5
3,2022.05.30,1238.5,2022-05-31,-17.5
2,2022.05.31,1248.0,2022-06-01,9.5
1,2022.06.02,1250.5,2022-06-03,2.5


In [9]:
# 한국 국고채 금리
list_df = []
for idx in range(50):
    pg = idx + 1
    url = f"https://finance.naver.com/marketindex/interestDailyQuote.naver?marketindexCd=IRR_GOVT03Y&page={pg}"
    headers = {
        "referer" : url,    
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
    }
    res = requests.get(url=url, headers=headers)
    list_df.append(pd.read_html(res.text)[0][["날짜", "파실 때"]])

df_kor_bond = pd.concat(list_df)
df_kor_bond["KOR_DT"] = df_kor_bond["날짜"].apply(lambda X: X.replace(".","-"))
df_kor_bond["KOR_DT"] = pd.to_datetime(df_kor_bond["KOR_DT"])
df_kor_bond["KOR_DT"] = df_kor_bond["KOR_DT"].apply(lambda X: X + datetime.timedelta(days=1))
df_kor_bond = df_kor_bond.rename(columns=({"파실 때": "kor_bond"}))

df_kor_bond = sort_by_date_diff_series_data(df_kor_bond, "kor_bond")
df_kor_bond = df_kor_bond[["KOR_DT", "kor_bond", "DIFF_kor_bond"]]
df_kor_bond

,KOR_DT,kor_bond,DIFF_kor_bond
5,2021-01-06,0.93,-0.02
4,2021-01-07,0.95,0.02
3,2021-01-08,0.96,0.01
2,2021-01-09,0.97,0.01
1,2021-01-12,0.96,-0.01
...,...,...,...
4,2022-05-28,2.94,-0.01
3,2022-05-31,2.92,-0.02
2,2022-06-01,3.02,0.10
1,2022-06-03,3.12,0.10


In [14]:
df_result = pd.merge(
    df_us_index, df_kodex, how="inner", on=["KOR_DT"]
)
df_result = pd.merge(
    df_result, df_usd, how="inner", on=["KOR_DT"]
)
df_result = pd.merge(
    df_result, df_kor_bond, how="inner", on=["KOR_DT"]
)

list_diff_columns = []
dict_rename_columns = {}
for col in df_result.keys():
    if "DIFF" in col:
        list_diff_columns.append(col)
        dict_rename_columns[col] = col.replace("DIFF_", "")
df_diff = df_result[["KOR_DT"] + list_diff_columns].rename(columns=dict_rename_columns)
df_diff

,KOR_DT,S&P500_adj_prc,Nasdaq_adj_prc,CrudeOil_adj_prc,Gold_adj_prc,10-YrBond_adj_prc,Closing_Price,Start_Price,High_Price,Low_Price,Volume,krw_usd,kor_bond
0,2021-06-08,-3.370117,67.229492,-0.389999,7.000000,0.009,-100.0,-185.0,-110.0,160.0,-4673209.0,-4.0,-0.02
1,2021-06-09,0.739746,43.190430,0.820000,-4.600098,-0.041,-610.0,-155.0,-320.0,-525.0,1421467.0,3.5,-0.03
2,2021-06-10,-7.709961,-13.160156,-0.090004,1.000000,-0.039,175.0,-525.0,-200.0,-75.0,-3116403.0,-2.5,-0.04
3,2021-06-11,19.630371,108.580078,0.330002,1.000000,-0.030,395.0,275.0,330.0,360.0,5712007.0,2.5,0.15
4,2021-06-15,7.709961,104.719727,-0.030006,-13.400024,0.039,135.0,0.0,160.0,140.0,2607051.0,2.5,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2022-05-24,72.389893,180.649414,-2.940002,6.000000,0.072,-665.0,-260.0,-175.0,-470.0,1763802.0,-8.5,0.00
180,2022-05-25,-32.270020,-270.819336,-0.520004,17.299927,-0.099,145.0,-295.0,-220.0,-45.0,-954968.0,0.5,-0.05
181,2022-05-26,37.250000,170.290039,0.560005,-18.900024,-0.011,-140.0,-90.0,130.0,-85.0,2959630.0,6.0,-0.02
182,2022-05-27,79.110107,305.910156,3.759995,1.200073,0.007,395.0,315.0,30.0,455.0,-1678204.0,-5.5,0.01


In [15]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

list_scale_columns = [
    "S&P500_adj_prc",
    "Nasdaq_adj_prc", 
    "CrudeOil_adj_prc", 
    "Gold_adj_prc", 
    "10-YrBond_adj_prc", 
    "krw_usd",
    "kor_bond",
]
list_value = [
    75.589844, 322.440430, 1.610001, 23.199951, -0.018, 2.5, 0.10, 
]

df_model = df_diff[df_diff["KOR_DT"] < "2022-06-03"]

X = df_model[list_scale_columns]
CP = df_model['Closing_Price']
HP = df_model['High_Price']
LP = df_model['Low_Price']

scaledX = scale.fit_transform(X)

regr = linear_model.LinearRegression()
regr.fit(scaledX, CP)

scaled = scale.transform([list_value])

print(regr.predict([scaled[0]]))

LinearRegression()

[418.23154915]


In [245]:
X = df_model[list_scale_columns]
y = df_model['Closing_Price']

regr = linear_model.LinearRegression()
regr.fit(X, y)

predictedCO2 = regr.predict([list_value])

print(predictedCO2)

LinearRegression()

[418.52722102]


In [171]:
df["krw_usd_diff"] = df["krw_usd"].diff()
df

,날짜,krw_usd,KOR_DT,diff,krw_usd_diff
9,2018.10.18,1135.5,2018-10-19,NaN,NaN
8,2018.10.19,1132.5,2018-10-20,-3.0,-3.0
7,2018.10.22,1132.0,2018-10-23,-0.5,-0.5
6,2018.10.23,1137.5,2018-10-24,5.5,5.5
5,2018.10.24,1134.0,2018-10-25,-3.5,-3.5
...,...,...,...,...,...
4,2022.05.27,1256.0,2022-05-28,-9.5,-9.5
3,2022.05.30,1238.5,2022-05-31,-17.5,-17.5
2,2022.05.31,1248.0,2022-06-01,9.5,9.5
1,2022.06.02,1250.5,2022-06-03,2.5,2.5


In [37]:
page_string = "&page="

def make_index_data(idx_url, idx_column, ren_column):
    list_df = []
    for idx in range(1):
        pg = idx + 1
        url = idx_url + f"{page_string}{pg}"
        headers = {
            "referer" : url,    
            "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
        }
        res = requests.get(url=url, headers=headers)
        try:
            list_df.append(pd.read_html(res.text, header=[1])[0][["날짜", idx_column]])
        except:
            list_df.append(pd.read_html(res.text)[0][["날짜", idx_column]])

    df_kor_idx = pd.concat(list_df)
    df_kor_idx["KOR_DT"] = df_kor_idx["날짜"].apply(lambda X: X.replace(".","-"))
    df_kor_idx["KOR_DT"] = pd.to_datetime(df_kor_idx["KOR_DT"])
    df_kor_idx["KOR_DT"] = df_kor_idx["KOR_DT"].apply(lambda X: X + datetime.timedelta(days=1))
    df_kor_idx = df_kor_idx.rename(columns=({idx_column: ren_column}))

    df_kor_idx = sort_by_date_diff_series_data(df_kor_idx, ren_column)
    df_kor_idx = df_kor_idx[["KOR_DT", ren_column, f"DIFF_{ren_column}"]]
    
    return df_kor_idx


dict_index_value = {
    "KRW_USD": ["https://finance.naver.com/marketindex/exchangeDailyQuote.naver?marketindexCd=FX_USDKRW", "매매기준율", "KRW_USD"],
    "KRW_EUR": ["https://finance.naver.com/marketindex/exchangeDailyQuote.naver?marketindexCd=FX_EURKRW", "매매기준율", "KRW_EUR"],
    "OIL_DU": ["https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_DU&fdtc=2", "파실 때", "OIL_DU"],
    "KR_INTEREST": ["https://finance.naver.com/marketindex/interestDailyQuote.naver?marketindexCd=IRR_GOVT03Y", "파실 때", "KR_INTEREST"],
    "KR_GOLD": ["https://finance.naver.com/marketindex/goldDailyQuote.naver?marketindexCd=goldDailyQuote", "매매기준율", "KR_GOLD"],
    "MATERIAL_COPPER": ["https://finance.naver.com/marketindex/worldDailyQuote.naver?fdtc=2&marketindexCd=CMDT_CDY", "파실 때", "COPPER"],
    "MATERIAL_LEAD": ["https://finance.naver.com/marketindex/worldDailyQuote.naver?fdtc=2&marketindexCd=CMDT_PDY", "파실 때", "LEAD"],
    "MATERIAL_GAS": ["https://finance.naver.com/marketindex/worldDailyQuote.naver?fdtc=2&marketindexCd=CMDT_NG", "파실 때", "NATURAL_GAS"],
    "MATERIAL_CORN": ["https://finance.naver.com/marketindex/worldDailyQuote.naver?fdtc=2&marketindexCd=CMDT_C", "파실 때", "CORN"],
}

df_material = pd.DataFrame(columns=["KOR_DT"])
for key, list_val in dict_index_value.items():
    df = make_index_data(list_val[0], list_val[1], list_val[2])
    if df_material.empty:
        df_material = df
    else:
        df_material = pd.merge(
            df_material, df, how="outer", on=["KOR_DT"]
        )
df_material.sort_values(by=["KOR_DT"])

,KOR_DT,KRW_USD,DIFF_KRW_USD,KRW_EUR,DIFF_KRW_EUR,OIL_DU,DIFF_OIL_DU,KR_INTEREST,DIFF_KR_INTEREST,KR_GOLD,DIFF_KR_GOLD,COPPER,DIFF_COPPER,LEAD,DIFF_LEAD,NATURAL_GAS,DIFF_NATURAL_GAS,CORN,DIFF_CORN
0,2022-05-25,1265.0,0.5,1354.56,4.45,NaN,NaN,NaN,NaN,75533.02,-129.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-05-26,1271.0,6.0,1355.01,0.45,NaN,NaN,NaN,NaN,75479.26,-53.76,9287.0,-173.5,2121.5,-60.0,NaN,NaN,NaN,NaN
2,2022-05-27,1265.5,-5.5,1356.24,1.23,NaN,NaN,NaN,NaN,75176.73,-302.53,9359.0,72.0,2106.0,-15.5,NaN,NaN,NaN,NaN
3,2022-05-28,1256.0,-9.5,1345.05,-11.19,110.86,1.08,2.94,-0.01,74845.11,-331.62,9383.0,24.0,2091.5,-14.5,8.73,-0.16,777.25,12.25
4,2022-05-31,1238.5,-17.5,1334.42,-10.63,NaN,NaN,2.92,-0.02,74023.91,-821.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-06-01,1248.0,9.5,1336.11,1.69,110.58,-0.28,3.02,0.10,73732.93,-290.98,9500.5,117.5,2176.0,84.5,8.14,-0.59,753.50,-23.75
9,2022-06-02,NaN,NaN,NaN,NaN,110.91,0.33,NaN,NaN,NaN,NaN,9455.0,-45.5,2143.5,-32.5,8.70,0.56,731.25,-22.25
6,2022-06-03,1250.5,2.5,1337.28,1.17,112.02,1.11,3.12,0.10,74447.27,714.34,NaN,NaN,NaN,NaN,8.48,-0.22,730.25,-1.00
7,2022-06-04,1252.0,1.5,1342.14,4.86,113.83,1.81,3.12,0.00,74579.42,132.15,NaN,NaN,NaN,NaN,8.52,0.04,727.00,-3.25
10,2022-06-07,NaN,NaN,NaN,NaN,114.01,0.18,NaN,NaN,NaN,NaN,9712.0,257.0,2181.0,37.5,9.32,0.80,742.50,15.50
